In [10]:
import numpy as np
import pandas as pd
import os
import math
from datetime import datetime, timedelta
from sklearn import linear_model

In [33]:
data_directory = "../Data/Study/A/"
intermediate = 'static/'
subdirectory = "shifted/"

In [19]:
def clean_df(df):
    result = df.loc[(df['Movement'] != "start") & (df['Movement'] != "transition") & (df['EyeTrackingEnabled'] == True) & (df['EyeCalibrationValid'] == True) & (df['EyeTrackingEnabledAndValid'] == True) & (df['EyeTrackingDataValid'] == True) & (df['GazeInputSource'] == "Eyes")].copy()
    #result = result[result['Movement'] != "transition"]
    #result = result[result['EyeTrackingDataValid'] == 'TRUE']
    #result = result[result['GazeInputSource'] == 'Eyes']
    result.reset_index(inplace=True, drop=True)
    return result

In [16]:
# def create_calibration_df(taskName):
#     frames = []
#     for file in fileList:
#         if taskName in file:
#             df = pd.read_csv(data_directory + subdirectory + file)
#             df = clean_df(df)
#             frames.append(df)
#     result = pd.concat(frames)
#     return result

In [13]:
def create_calibration_df(taskName, fileList, participant):
    frames = []
    for file in fileList:
        if taskName in file:
            filePath = data_directory + participant + '/' + subdirectory + file
            df = pd.read_csv(filePath)
            df = clean_df(df)
            frames.append(df)
    if len(frames) == 0:
        print("error: task not found")
        return None
    else:
        result = pd.concat(frames)
        return result

In [14]:
def calculate_regression_coefficients(df):
    x1 = df[['GazeAngleX','GazeAngleY']]
    y1 = df['TargetAngleX']
    x2 = df[['GazeAngleX','GazeAngleY']]
    y2 = df['TargetAngleY']
    x_regr = linear_model.LinearRegression()
    x_regr.fit(x1.to_numpy(), y1.to_numpy())
    y_regr = linear_model.LinearRegression()
    y_regr.fit(x2.to_numpy(), y2.to_numpy())
    return [x_regr, y_regr]

In [21]:
def calibrate_files(taskname):
    output_subdirectory = intermediate + "recalibrated_" + taskname
    participantList = os.listdir(data_directory)
    for participant in participantList:
        print(participant)
        if participant == '.DS_Store':
            print("skipping calibration")
            continue
        if os.path.exists(data_directory + participant + '/' + output_subdirectory):
            print("already calibrated")
            continue
        fileList = os.listdir(data_directory + participant + '/' + intermediate + subdirectory)
        calibration_df = create_calibration_df(taskname, fileList, participant)
        [x_coeff, y_coeff] = calculate_regression_coefficients(calibration_df)
        for file in fileList:
            filePath = data_directory + participant + '/' + intermediate + subdirectory + file
            if '.csv' in file:
                df = pd.read_csv(filePath)
                df = clean_df(df)
                df['CalibratedGazeAngleX'] = df.apply(lambda row: x_coeff.predict([[row['GazeAngleX'],row['GazeAngleY']]])[0], axis = 1)
                df['CalibratedGazeAngleY'] = df.apply(lambda row: y_coeff.predict([[row['GazeAngleX'],row['GazeAngleY']]])[0], axis = 1)
                recalibrated_df_output_directory = data_directory + participant + '/' + output_subdirectory
                recalibrated_df_output_path = data_directory + participant + '/' + output_subdirectory + '/' + file
                if not os.path.exists(recalibrated_df_output_directory):
                    os.mkdir(recalibrated_df_output_directory)
                #filePrefix = file.split('.')[0]
                #recalibrated_df_output_path = data_directory + output_subdirectory + '/' + filePrefix + '_recalibrated_' + taskname + '.csv'
                df.to_csv(recalibrated_df_output_path, index=False)

In [21]:
# def calibrate_files(taskname, output_subdirectory):
#     calibration_df = create_calibration_df(taskname)
#     [x_coeff, y_coeff] = calculate_regression_coefficients(calibration_df)
#     for file in fileList:
#         if '.csv' in file:
#             df = pd.read_csv(data_directory + subdirectory + file)
#             df = clean_df(df)
#             df['CalibratedGazeAngleX'] = df.apply(lambda row: x_coeff.predict([[row['GazeAngleX'],row['GazeAngleY']]])[0], axis = 1)
#             df['CalibratedGazeAngleY'] = df.apply(lambda row: y_coeff.predict([[row['GazeAngleX'],row['GazeAngleY']]])[0], axis = 1)
#             if not os.path.exists(os.path.join(data_directory, output_subdirectory)):
#                 os.mkdir(os.path.join(data_directory, output_subdirectory))
#             filePrefix = file.split('.')[0]
#             #recalibrated_df_output_path = data_directory + output_subdirectory + '/' + filePrefix + '_recalibrated_' + taskname + '.csv'
#             recalibrated_df_output_path = data_directory + output_subdirectory + '/' + file
#             df.to_csv(recalibrated_df_output_path)

In [34]:
calibrate_files("calibration")

03
already calibrated
04
already calibrated
32
34
05
already calibrated
02
.DS_Store
skipping calibration
18
27
29
10
19
43


FileNotFoundError: [Errno 2] No such file or directory: '../Data/Study/A/43/static/shifted/'